In [1]:
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import sqlite3 as sql

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.6.0'

In [4]:
# employee = pd.read_csv('employee.csv')
# employee.head()

con = sql.connect("data/hr.sqlite")
# con.row_factory = sql.Row
# cur = con.cursor()
employee = pd.read_sql_query("select is_married, is_diversity_job_fair, employee_salary, is_active, is_terminated, is_underrep_gender, is_citizen, gender_id, is_hispanic_latino, is_underrep_race_eth, survey_engagement_score, survey_emp_satisfaction_score, count_special_projects, count_days_late_past_30_days, count_absences, employee_status from employee", con)

# dataset = cur.fetchall()
con.close()


In [5]:
employee

,is_married,is_diversity_job_fair,employee_salary,is_active,is_terminated,is_underrep_gender,is_citizen,gender_id,is_hispanic_latino,is_underrep_race_eth,survey_engagement_score,survey_emp_satisfaction_score,count_special_projects,count_days_late_past_30_days,count_absences,employee_status
0,0,0,65288,1,0,1,1,1,0,0,3.19,3.0,0,0,9,Active
1,0,0,66738,1,0,0,1,0,0,0,4.53,5.0,0,0,5,Active
2,1,0,50428,0,1,1,1,1,0,1,5.00,3.0,0,0,11,Voluntarily Terminated
3,1,0,75281,0,1,1,1,1,0,0,5.00,3.0,5,0,11,Voluntarily Terminated
4,1,1,63000,1,0,0,1,0,0,1,4.50,2.0,6,0,14,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,0,0,58370,1,0,0,1,0,0,1,3.69,3.0,0,0,18,Active
308,0,0,59231,1,0,0,1,0,1,0,2.30,1.0,0,2,17,Active
309,1,1,54670,0,1,0,1,0,1,1,4.20,4.0,0,0,12,Voluntarily Terminated
310,0,0,52505,0,1,1,1,1,0,0,5.00,5.0,0,0,1,Voluntarily Terminated


In [6]:

X = employee.drop("employee_status", axis=1)
y = employee["employee_status"]
print(X.shape, y.shape)

(312, 15) (312,)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [9]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [11]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=15 ))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1600      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 12,003
Trainable params: 12,003
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=30,
    shuffle=True,
    verbose=2
)

Epoch 1/60
8/8 - 0s - loss: 0.8055 - accuracy: 0.7949
Epoch 2/60
8/8 - 0s - loss: 0.6012 - accuracy: 0.7009
Epoch 3/60
8/8 - 0s - loss: 0.4589 - accuracy: 0.8590
Epoch 4/60
8/8 - 0s - loss: 0.3390 - accuracy: 0.9530
Epoch 5/60
8/8 - 0s - loss: 0.2507 - accuracy: 0.9530
Epoch 6/60
8/8 - 0s - loss: 0.1951 - accuracy: 0.9530
Epoch 7/60
8/8 - 0s - loss: 0.1659 - accuracy: 0.9530
Epoch 8/60
8/8 - 0s - loss: 0.1474 - accuracy: 0.9530
Epoch 9/60
8/8 - 0s - loss: 0.1364 - accuracy: 0.9530
Epoch 10/60
8/8 - 0s - loss: 0.1304 - accuracy: 0.9530
Epoch 11/60
8/8 - 0s - loss: 0.1249 - accuracy: 0.9530
Epoch 12/60
8/8 - 0s - loss: 0.1186 - accuracy: 0.9530
Epoch 13/60
8/8 - 0s - loss: 0.1161 - accuracy: 0.9530
Epoch 14/60
8/8 - 0s - loss: 0.1127 - accuracy: 0.9573
Epoch 15/60
8/8 - 0s - loss: 0.1091 - accuracy: 0.9573
Epoch 16/60
8/8 - 0s - loss: 0.1098 - accuracy: 0.9573
Epoch 17/60
8/8 - 0s - loss: 0.1076 - accuracy: 0.9573
Epoch 18/60
8/8 - 0s - loss: 0.1042 - accuracy: 0.9573
Epoch 19/60
8/8 - 0

In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

3/3 - 0s - loss: nan - accuracy: 0.9359
Normal Neural Network - Loss: nan, Accuracy: 0.9358974099159241


In [18]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [19]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

NameError: name 'prediction_labels' is not defined

In [20]:
# con = sql.connect("data/hr.sqlite")
# con.row_factory = sql.Row
# cur = con.cursor()
# cur.execute("select is_married, gender_id, is_hispanic_latino,survey_engagement_score, count_absences, employee_status from employee")
# dataset = cur.fetchall()